# Backtest and Optimiser !!!
1. This script can backtest and optimise any single position strategy.
2. The script uses a csv from Binance spot BTC/USDT, please ensure you have a suitable CSV before attempting to read in a dataframe.
3. When testing/optimising your strategy, it is a good idea to mask the data to a smaller sample set to confirm accuracy before completing a full backtest. As a rough example to optimise 5 timeframes, with two moving averages, and four fixed TP and SL options i.e 740 iterations of Backtest() could take over 8 hours!

In [1]:
#Libraries
import pandas as pd
import pandas_ta as ta
import numpy as np
import itertools
#import plotly.graph_objects as go

In [53]:
#READING AND CLEANING + SET INDEX
df = pd.read_csv("../Binance_BTCUSDT_1min.csv")
df = df.iloc[:,:6]
df.columns=['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.reset_index(drop=True, inplace=True)
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date")
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...
2023-06-13 16:46:00,25848.59,25852.45,25843.93,25850.01,87.557760
2023-06-13 16:47:00,25850.00,25866.98,25850.00,25866.97,50.890100
2023-06-13 16:48:00,25866.98,25866.98,25855.67,25863.11,53.087330


In [54]:
#MASKING TO DECREASE SAMPLE SIZE
start_date = "2022-01-01 00:00:00"
end_date = "2024-01-01 00:00:00"
mask = (df.index > start_date) & (df.index <= end_date)
df=df.loc[mask]
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-01 00:01:00,46250.00,46344.23,46234.39,46312.76,42.38106
2022-01-01 00:02:00,46312.76,46381.69,46292.75,46368.73,51.29955
2022-01-01 00:03:00,46368.73,46391.49,46314.26,46331.08,30.45894
2022-01-01 00:04:00,46331.07,46336.10,46300.00,46321.34,20.96029
2022-01-01 00:05:00,46321.34,46443.56,46280.00,46436.03,35.86682
...,...,...,...,...,...
2023-06-13 16:46:00,25848.59,25852.45,25843.93,25850.01,87.55776
2023-06-13 16:47:00,25850.00,25866.98,25850.00,25866.97,50.89010
2023-06-13 16:48:00,25866.98,25866.98,25855.67,25863.11,53.08733


In [55]:
#Function for resampling the dataframe
def resample_df(df, freq):
    resampled_open = df.Open.resample(freq).first()
    resampled_high = df.High.resample(freq).max()
    resampled_low = df.Low.resample(freq).min()
    resampled_close = df.Close.resample(freq).last()
    resampled_volume = df.Volume.resample(freq).sum()
    new_df = pd.concat([resampled_open, resampled_high, resampled_low, resampled_close, resampled_volume], axis=1)
    new_df.dropna(inplace=True)
    return new_df

In [56]:
def calc_ma(new_df, x, y, z):
    new_df['fastSMA'] = new_df.Close.rolling(x).mean()
    new_df['midSMA'] = new_df.Close.rolling(y).mean()
    new_df['slowSMA'] = new_df.Close.rolling(z).mean()

In [57]:
def calc_price(new_df):
    new_df["price"] = new_df.Open.shift(-1)

In [58]:
def calc_buy_signal(new_df):
    new_df["buy_signal"] = np.where(((new_df.Close<new_df.fastSMA) & (new_df.Close.shift(1)>new_df.fastSMA.shift(1)) & (new_df.midSMA>new_df.slowSMA)), True, False)

In [59]:
new_df = resample_df(df,"1D")

In [60]:
calc_ma(new_df, 200, 20, 50)

In [61]:
calc_buy_signal(new_df)

In [62]:
new_df

,Open,High,Low,Close,Volume,fastSMA,midSMA,slowSMA,buy_signal
Date,,,,,,,,,
2022-01-01,46250.00,47954.63,46234.39,47722.65,19563.88751,NaN,NaN,NaN,False
2022-01-02,47722.66,47990.00,46654.00,47286.18,18340.46040,NaN,NaN,NaN,False
2022-01-03,47286.18,47570.00,45696.00,46446.10,27662.07710,NaN,NaN,NaN,False
2022-01-04,46446.10,47557.54,45500.00,45832.01,35491.41360,NaN,NaN,NaN,False
2022-01-05,45832.01,47070.00,42500.00,43451.13,51784.11857,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
2023-06-09,26498.62,26783.33,26269.91,26477.81,27934.70970,23442.56090,26920.2395,27533.8894,False
2023-06-10,26477.80,26533.87,25358.00,25841.21,64944.60108,23490.63225,26874.9110,27505.4568,False
2023-06-11,25841.22,26206.88,25634.70,25925.55,30014.29595,23537.24445,26828.7250,27467.6308,False


In [63]:
len(new_df[new_df.buy_signal>0])

0

In [64]:
new_df.dropna(inplace=True)

In [ ]:
#ADDING COLUMNS AND SIGNALS
#adx = ta.adx(df.High, df.Low, df.Close, length=9)
#df['ADX'] = adx.ADX_9
#df['RSI'] = ta.rsi(df.Close, length=9)
#df['CCI'] = ta.cci(df.High, df.Low, df.Close, length=9)
#df['VWAP'] = ta.vwap(df.High, df.Low, df.Close, df.Volume, anchor="M")
#df["SMA_200"] = ta.sma(df.Close, 200)
#df["SMA_50"] = ta.sma(df.Close, 50)
#df["lookBackLow"] = df.Low.rolling(2).min()
#df.dropna(inplace=True)
#df

#calc_indicators functions
#def calc_vwap(new_df, anchor_period):
    #new_df['VWAP'] = ta.vwap(new_df.High, new_df.Low, new_df.Close, new_df.Volume, anchor=anchor_period)

#def calc_vwap(new_df, anchor_period):
    #new_df['VWAP'] = ta.vwap(new_df.High, new_df.Low, new_df.Close, new_df.Volume, anchor=anchor_period)
    
#def calc_ma(new_df, n):
    #new_df['SMA'] = new_df.Close.rolling(n).mean()
    
#def calc_fast_rsi(new_df, frsi):
    #new_df["fast_rsi"] = ta.rsi(new_df.Close, length=frsi)
    
    #TO RUN CALC
    #calc_vwap(new_df, anchor_period)
    #calc_fast_ma(new_df, m)
    #calc_slow_ma(new_df,n)

#def calc_pivots(new_df, lb):
    #new_df["min_val"] = df.Low.rolling(lb).min()
    #new_df["max_val"] = df.High.rolling(lb).max()

#def calc_rsi(new_df, rsi_length, rsiSMA_period):
    #new_df["rsi"] = ta.rsi(new_df.Close, length=rsi_length).round(2)
    #new_df["rsiSMA"] = new_df.rsi.rolling(rsiSMA_period).mean().round(2)
    #new_df['rsi_std'] = new_df.rsi.rolling(20).std(ddof=0)*2
    #new_df["rsibbUpper"] = (new_df.rsiSMA + new_df.rsi_std).round(2)
    #new_df["rsibbLower"] = (new_df.rsiSMA - new_df.rsi_std).round(2)

In [71]:
def backtest(df, freq, x, y, z, tp, sl):

    new_df = resample_df(df, freq)
    calc_ma(new_df, x, y, z)
    calc_price(new_df)
    calc_buy_signal(new_df)
    new_df.dropna(inplace=True)
    
    #Error handle for no buy signals
    if len(new_df[new_df.buy_signal > 0]) < 1:
        empty_result = pd.DataFrame({
            "entry_time": [0],
            "entry_price": [0],
            "tp_target": [0],
            "sl_target": [0],
            "exit_time": [0],
            "exit_price": [0],
            "pnl": [0]
        })
        amount = 0
        winrate = 0
        pnl = 0
        return amount, winrate, pnl
        return empty_result
    
    in_position = False
    trades = []
    current_trade = {}
            
    for i in range(len(new_df)-1):
    #Check exit conditions
        if in_position:
            if new_df.iloc[i].Low < current_trade["sl_price"]:
                current_trade["exit_price"] = current_trade["sl_price"]
                pnl = (current_trade["exit_price"] - current_trade["entry_price"]) / current_trade["entry_price"]
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "sl_target":current_trade["sl_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["sl_price"],
                    "pnl": pnl
                })
                current_trade = {}
                in_position = False

            elif new_df.iloc[i].High > current_trade["tp_price"]:
                current_trade["exit_price"] = current_trade["tp_price"]
                pnl = (current_trade["exit_price"] - current_trade["entry_price"]) / current_trade["entry_price"]
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "sl_target":current_trade["sl_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["tp_price"],
                    "pnl":pnl,
                })
                current_trade = {}
                in_position = False

        #Check entry conditions
        if not in_position:
            if new_df.iloc[i].buy_signal == True:
                current_trade["entry_price"] = new_df.iloc[i].price
                current_trade["entry_time"] = new_df.iloc[i+1].name
                current_trade["tp_price"] = new_df.iloc[i].price*tp
                current_trade["sl_price"] = new_df.iloc[i].price*sl
                in_position = True
                
    data = pd.DataFrame(trades)
    amount = len(data)
    winrate = len(data.loc[data.pnl.values>0])/len(data)*100
    pnl = sum(pd.Series(data.pnl))
    return amount, winrate, pnl
    return pd.DataFrame(trades)

In [72]:
#backtest(df, freq, rsi_length, rsiSMA_period, overSold, tp, sl):
#1H	7	20	20	1.02	0.97

In [73]:
backtest(df, "1D", 200, 20, 50, 1.02, 0.97)

(0, 0, 0)

# Optimiser

In [74]:
# Define the parameter combinations
freq_values = ["4H", "1D"]
x_values = [20, 50, 200]
y_values = [20, 50, 200]
z_values = [20, 50, 200]
tp_values = [1.02, 1.03]
sl_values = [0.98, 0.97]

In [ ]:
#backtest(df, freq, rsi_length, rsiSMA_period, overSold, tp, sl):

In [76]:
#Create an empty dataframe
results_df = pd.DataFrame(columns=["freq", "fastSMA_length", "midSMA_length", "slowSMA_length", "tp", "sl", "amount", "winrate", "pnl"])

In [77]:
for freq, x, y, z, tp, sl in itertools.product(freq_values, x_values, y_values, z_values, tp_values, sl_values):
    amount, winrate, pnl = backtest(df, freq, x, y, z, tp, sl)
    result = pd.DataFrame([[freq, x, y, z, tp, sl, amount, winrate, pnl]], columns=["freq", "fastSMA_length", "midSMA_length", "slowSMA_length", "tp", "sl", "amount", "winrate", "pnl"])
    results_df = pd.concat([results_df, result], ignore_index=True)

# Print the results DataFrame
print(results_df)

    freq fastSMA_length midSMA_length slowSMA_length    tp    sl amount  \
0     4H             20            20             20  1.02  0.98      0   
1     4H             20            20             20  1.02  0.97      0   
2     4H             20            20             20  1.03  0.98      0   
3     4H             20            20             20  1.03  0.97      0   
4     4H             20            20             50  1.02  0.98     72   
..   ...            ...           ...            ...   ...   ...    ...   
211   1D            200           200             50  1.03  0.97      0   
212   1D            200           200            200  1.02  0.98      0   
213   1D            200           200            200  1.02  0.97      0   
214   1D            200           200            200  1.03  0.98      0   
215   1D            200           200            200  1.03  0.97      0   

       winrate   pnl  
0            0     0  
1            0     0  
2            0     0  
3      

In [78]:
results_df

,freq,fastSMA_length,midSMA_length,slowSMA_length,tp,sl,amount,winrate,pnl
0,4H,20,20,20,1.02,0.98,0,0,0
1,4H,20,20,20,1.02,0.97,0,0,0
2,4H,20,20,20,1.03,0.98,0,0,0
3,4H,20,20,20,1.03,0.97,0,0,0
4,4H,20,20,50,1.02,0.98,72,45.833333,-0.12
...,...,...,...,...,...,...,...,...,...
211,1D,200,200,50,1.03,0.97,0,0,0
212,1D,200,200,200,1.02,0.98,0,0,0
213,1D,200,200,200,1.02,0.97,0,0,0
214,1D,200,200,200,1.03,0.98,0,0,0


In [ ]:
#results_df[results_df.pnl.max()]
results_df["pnl1"] = results_df.pnl.round(2)

In [ ]:
filtered_results = results_df[(results_df["winrate"] > 50) & (results_df["amount"] > 300) & (results_df["pnl1"] > 1.5) & (results_df["sl"]>0.95)]
filtered_results

In [ ]:
results_df[results_df.pnl>3]

In [ ]:
#results_df.to_csv('BTCUSDT_RSIBB_BREAKOUT-Backtest.csv', index=True)

In [ ]:
pd.read_csv('BTCUSDT_RSIBB_BREAKOUT-Backtest.csv')

# Visualisation

In [ ]:
#CANDLE STICK AND MERGED PLOTS
fig = go.Figure(data=[go.Candlestick(
    x = new_df.index,
    open = new_df.Open,
    high = new_df.High,
    low = new_df.Low,
    close = new_df.Close,
    increasing_line_color = "rgba(0, 0, 0, 0.5)",
    decreasing_line_color = "rgba(0, 0, 0, 0.5)",
    name="Candlesticks"
    ),
    go.Scatter(x=new_df.index, y=new_df.VWAP, line=dict(color='red', width=1)),
    go.Scatter(x=new_df.index, y=new_df.SMA, line=dict(color='blue', width=1))
])

#ENTRY PLOTS
fig.add_trace(go.Scatter(
    x=trades.entry_time,
    y=trades.entry_price,
    mode = "markers",
    customdata=trades,
    marker_symbol="triangle-up",
    marker_color="rgba(0, 255, 0, 0.9)",
    marker_line_color="rgba(0, 0, 0, 0.5)",
    marker_size =13,
    marker_line_width=1,
    name="Entries",
    hovertemplate="Entry Time: %{customdata[1]}<br>"
        "Entry Price: %{y:.2f}<br>"
        "Size: %{customdata[2]:.6f}"
))

#EXIT PLOTS
fig.add_trace(go.Scatter(
    x=trades.exit_time,
    y=trades.exit_price,
    mode = "markers",
    customdata=trades,
    marker_symbol="triangle-down",
    marker_color="rgba(255, 0, 0, 0.9)",
    marker_line_color="rgba(0, 0, 0, 0.5)",
    marker_size =13,
    marker_line_width=1,
    name="Exits",
    hovertemplate="Exit Time: %{customdata[4]}<br>"
        "Exit Price: %{y:.2f}<br>"
        #"Size: %{customdata[2]:.6f}"
))

#fig.update_layout (xaxis_rangeslider_visable = False)
fig